In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Running Optimizer from command line

One more time we request our sales to visit all customers, so again a TSP-example. You have already seen the code a few times, so this time we place the OptimizerModel in a separate file. You can check the code  [here](TSPModel.py).   

In [ ]:
from dadk.Optimizer import *
from TSPModel import TSPModel

optimizer = Optimizer(TSPModel())

We have stored a 'dao'-file for you, with 12 cities on a random map. To reload the stored solution without additional annealing, use the parameter `immediate_load=True`

In [ ]:
replay = Optimizer(TSPModel(persistent_file='random_12.dao'), immediate_load=True, read_only=True)

By adding the parameter `batch_mode=True` the Optimizer framework will not generate the GUI elements.

In [ ]:
replay_txt = Optimizer(TSPModel(persistent_file='random_12.dao'), immediate_load=True, read_only=True, batch_mode=True)

If you use the parameter `immediate_run=True` instead of `immediate_load=True` the Optimizer will run again through all steps.

In [ ]:
execute = Optimizer(TSPModel(persistent_file='random_12.dao'), immediate_run=True, read_only=True, batch_mode=True)

Let's check what is in the directory. 

| type | size | map | solution |
| --- | --- | --- | --- |
| random | 12 | [map](random_12_map.png) | [route](random_12_route.png) |

If you run the model with a GUI, the initial map and solution are shown on the tabs. With `batch_mode=True` the plots are redirected to files. This is not an automatic feature. Instead, the developer of the model has to modify the code, so that in both situations (with and without GUI) the user can find the result.     

```python
    # for example show plot in GUI mode and save to file in batch-mode
    if self.Optimizer.batch_mode:
        plt.savefig(self.persistent_file.replace('.dao', '_map.png')) 
    else:
        plt.show()
```

In this case it might be a good idea to create filenames dynamically based on the selected `persistent_file`, like in the code example above.

In [ ]:
!dir random_12*

One way to run your model with different combinations of input parameters in batch mode, instead of manually clicking through the GUI for each combination of input parameters, would be to create a dedicated 'dao'-file for each combination.    

In [ ]:
import json

for N in [4, 6, 8]:
    
    filename = f"random__{N}.dao"
    
    data = {
        "parameter_definition":{
            "load_parameter": {
                "N":N,
                "special_case":False,
                "random_seed":42
            }            
        }
    }
    
    with open(filename, mode="w") as fp:
        json.dump(data, fp, indent=4, sort_keys=True)

    execute = Optimizer(TSPModel(persistent_file=filename), immediate_run=True, batch_mode=True)

You will find the solutions again in the directory.

| type | size | map | solution |
| --- | --- | --- | --- |
| random |  4 | [map](random__4_map.png) | [route](random__4_route.png) |
| random |  6 | [map](random__6_map.png) | [route](random__6_route.png) |
| random |  8 | [map](random__8_map.png) | [route](random__8_route.png) |
| random | 12 | [map](random_12_map.png) | [route](random_12_route.png) |

In [ ]:
!dir random*

This approach has a few problems:
* everything is running in a Jupyter notebook
* everything is running sequentially in a single python environment
* all output exists only in the notebook
* no simple parallelization
* no simple scale out 

Instead, you can request your generated `Optimizer`-instance to generate a batch-script wrapper for you.

In [ ]:
with open('BatchTSP.py', mode='w', encoding='utf-8') as fp:
    fp.write(optimizer.create_batch())

With the wrapper one can set ALL parameters available in the GUI. Let's check the wrapper build-in help function first.

In [ ]:
!python BatchTSP.py --help 2>&1

Let's run the model again, this time for 12 cities on a circle.

In [ ]:
!python BatchTSP.py circle_12.dao --special_case True --N 12

You will find the solutions again in the directory.

| type | size | map | solution |
| --- | --- | --- | --- |
| circle | 12 | [map](circle_12_map.png) | [route](circle_12_route.png) |

In [ ]:
!dir circle_12*

Let's create a small Windows Command script `run.bat`, and direct echo output in a separate log-file. 

In [ ]:
with open('run.bat', mode='w') as fp:
    for N in [4, 6, 8]:
        fp.write(f"python BatchTSP.py circle__{N}.dao --special_case True --N {N} 2>&1 > circle__{N}.log\n")

And now we execute this Command script.

In [ ]:
!run.bat

You will find the solutions again in the directory.

| type | size | map | solution | log | 
| --- | --- | --- | --- | --- | 
| circle |  4 | [map](circle__4_map.png) | [route](circle__4_route.png) | [log](circle__4.log) |
| circle |  6 | [map](circle__6_map.png) | [route](circle__6_route.png) | [log](circle__6.log) |
| circle |  8 | [map](circle__8_map.png) | [route](circle__8_route.png) | [log](circle__8.log) |
| circle | 12 | [map](circle_12_map.png) | [route](circle_12_route.png) |                      |

In [ ]:
!dir circle*